In [0]:
df = spark.read.table('sleeper.bronze_players')

In [0]:
display(df)

In [0]:
%sql

SELECT * FROM sleeper.bronze_rosters

In [0]:
from pyspark.sql.functions import current_timestamp, expr, explode

In [0]:
df = spark.read.table("sleeper.bronze_rosters") \
    .withColumn("player_id", explode("players")) \
    .withColumn("is_starter", expr("array_contains(starters, player_id)")) \
    .withColumn("player_nickname", expr("metadata['p_nick_' || player_id]"))

df = df.select("owner_id", "roster_id", "player_id", "is_starter", "player_nickname", "_league_id", "_matchup_week", "_year", "_ingested_ts")\
    .withColumn("_snapshot_ts", current_timestamp())
display(df)

In [0]:
%sql

SELECT * FROM sleeper.bronze_rosters

In [0]:
df = spark.read.table("sleeper.bronze_rosters") \
    .withColumn("streak", expr("metadata['streak']")) \
    .withColumn("record", expr("metadata['record']"))\
    .withColumn("wins", expr("settings['wins']"))\
    .withColumn("losses", expr("settings['losses']"))\
    .withColumn("ties", expr("settings['ties']"))\
    .withColumn("fpts", expr("settings['fpts'] + settings['fpts_decimal'] / 100")) \
    .withColumn("fpts_against", expr("settings['fpts_against'] + settings['fpts_against_decimal'] / 100")) \
    .withColumn("total_moves", expr("settings['total_moves']")) \
    .withColumn("waiver_budget_used", expr("settings['waiver_budget_used']")) \
    .withColumn("waiver_position", expr("settings['waiver_position']"))

df = df.select(
    "owner_id", 
    "roster_id",
    "streak",
    "record",
    "wins",
    "losses",
    "ties",
    "fpts",
    "fpts_against",
    "total_moves",
    "waiver_budget_used",
    "waiver_position",
    "_league_id",
    "_matchup_week",
    "_year",
    "_ingested_ts",
)

display(df)

In [0]:
%sql

SELECT * FROM sleeper.bronze_matchups

In [0]:
df = spark.read.table('sleeper.bronze_matchups')\
    .select(
        "matchup_id",
        "roster_id",
        "points",
        "_league_id",
        "_matchup_week",
        "_year",
        "_ingested_ts"
    ).withColumn("_snapshot_ts", current_timestamp())

In [0]:
%sql

SELECT * FROM sleeper.bronze_matchups

In [0]:
from pyspark.sql.functions import array_contains, col, explode

df = spark.read.table('sleeper.bronze_matchups') \
    .withColumn("player_id", explode(col("players"))) \
    .withColumn("is_starter", array_contains(col("starters"), col("player_id"))) \
    .withColumn("player_points", col("players_points")[col("player_id")])

df = df.select(
    "roster_id",
    "matchup_id",
    "player_id",
    "player_points",
    "is_starter",
    "_league_id",
    "_matchup_week",
    "_year",
    "_ingested_ts",
)

display(df)

In [0]:
%sql

SELECT * FROM sleeper.bronze_users

In [0]:
df = spark.read.table('sleeper.bronze_users')\
    .withColumnRenamed("display_name", "owner_name")\
    .withColumnRenamed("user_id", "owner_id")\
    .withColumnRenamed("is_owner", "is_commissioner")\
    .withColumn("team_name", col("metadata.team_name"))

df = df.select(
    "owner_id",
    "owner_name",
    "is_bot",
    "is_commissioner",
    "team_name",
    "_league_id",
    "_matchup_week",
    "_year",
    "_ingested_ts"
)

display(df)

In [0]:
%sql

SELECT * FROM sleeper.silver_matchups_players_dim

In [0]:
from pyspark.sql.functions import col, concat_ws, coalesce, when
from pyspark.sql.window import Window
import pyspark.sql.functions as F

df_matchups_players_dim = spark.read.table('sleeper.silver_matchups_players_dim')
df_players_dim = spark.read.table('sleeper.silver_players_dim')

df_players_dim = df_players_dim.select(
    "player_id",
    "_league_id",
    "_matchup_week",
    coalesce(col("full_name"), col("last_name")).alias("player_name"),
    col("position").alias("player_position"),
    col("team").alias("nfl_team"),
    "years_exp",
    "injury_status",
    concat_ws(" ", col("injury_body_part"), col("injury_notes")).alias("injury_notes"),
    "college",
    when(col("years_exp") == 1, True).otherwise(False).alias("is_rookie")
)

df_joined = df_matchups_players_dim.join(
    df_players_dim,
    (df_matchups_players_dim.player_id == df_players_dim.player_id) &
    (df_matchups_players_dim._league_id == df_players_dim._league_id) &
    (df_matchups_players_dim._matchup_week == df_players_dim._matchup_week)
)

window_spec = Window.partitionBy(
    df_matchups_players_dim["_league_id"],
    df_matchups_players_dim["_matchup_week"],
    "player_position"
).orderBy("player_points")

df_joined = df_joined.withColumn("position_points_percentile", F.percent_rank().over(window_spec))

df_joined = df_joined.select(
    "roster_id",
    "matchup_id",
    df_matchups_players_dim.player_id,
    "player_name",
    "player_position",
    "nfl_team",
    "player_points",
    "position_points_percentile",
    "is_starter",
    "years_exp",
    "is_rookie",
    "injury_status",
    "injury_notes",
    "college",
    df_matchups_players_dim._league_id,
    df_matchups_players_dim._matchup_week,
    df_matchups_players_dim._year
)

display(df_joined)

In [0]:
%sql

SELECT * FROM sleeper.silver_matchups_fact

In [0]:
df_matchups_fact = spark.read.table('sleeper.silver_matchups_fact')
df_rosters_dim = spark.read.table('sleeper.silver_rosters_dim')
df_users_dim = spark.read.table('sleeper.silver_users_dim')

df_result = df_matchups_fact.join(
    df_rosters_dim,
    (df_matchups_fact._league_id == df_rosters_dim._league_id) &
    (df_matchups_fact.roster_id == df_rosters_dim.roster_id) &
    (df_matchups_fact._matchup_week == df_rosters_dim._matchup_week)
).join(
    df_users_dim,
    (df_matchups_fact._league_id == df_users_dim._league_id) &
    (df_rosters_dim.owner_id == df_users_dim.owner_id) &
    (df_matchups_fact._matchup_week == df_users_dim._matchup_week)
).select(
    "matchup_id",
    df_matchups_fact.roster_id,
    df_users_dim.owner_id,
    df_users_dim.owner_name,
    df_users_dim.is_commissioner,
    df_users_dim.team_name,
    df_matchups_fact.points,
    df_rosters_dim.streak,
    df_rosters_dim.record,
    df_rosters_dim.wins,
    df_rosters_dim.losses,
    df_rosters_dim.ties,
    df_rosters_dim.fpts,
    df_rosters_dim.fpts_against,
    df_rosters_dim.waiver_budget_used,
    df_rosters_dim.waiver_position,
    df_matchups_fact._league_id,
    df_matchups_fact._matchup_week,
    df_matchups_fact._year
)

display(df_result)

In [0]:
%sql

SELECT * FROM sleeper.silver_model_roster_results

In [0]:
%sql

SELECT * FROM sleeper.silver_model_player_performances

golden copy

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, MapType, DoubleType, NullType, FloatType, BooleanType, TimestampType, LongType
from pyspark.sql.functions import col, expr, explode, current_timestamp, sum, when, lit, array_contains, coalesce, concat_ws
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [0]:
df_roster_results = spark.read.table('sleeper.silver_model_roster_results')
df_player_performances = spark.read.table('sleeper.silver_model_player_performances')

# join rosters with player information to get complete breakdown of the matchup week
df_joined_results = df_roster_results.alias("roster").join(
    df_player_performances.alias("performance"),
    (col("roster._league_id") == col("performance._league_id")) &
    (col("roster.roster_id") == col("performance.roster_id")) &
    (col("roster.matchup_id") == col("performance.matchup_id")) &
    (col("roster._matchup_week") == col("performance._matchup_week"))
)

# aggregate the data at a roster level to get total amount of starter and roster points, to compute roster strength
df_aggregated = df_joined_results.groupBy(
    "roster._league_id", "roster._matchup_week","roster.matchup_id", "roster.roster_id", "roster.owner_id", "roster.owner_name",
    "roster.is_commissioner", "roster.team_name", "roster.matchup_points", "roster.team_streak",
    "roster.team_record", "roster.team_total_wins", "roster.team_total_losses", "roster.team_total_ties",
    "roster.team_total_fpts", "roster.team_total_fpts_against", "roster.team_waiver_budget_used",
    "roster.team_waiver_position", "roster._year"
).agg(
    F.sum(F.when(col("performance.is_starter"), col("performance.player_points")).otherwise(0)).alias("starter_points"),
    F.sum(F.when(~col("performance.is_starter"), col("performance.player_points")).otherwise(0)).alias("bench_points")
).withColumn(
    "roster_strength", col("starter_points") + (0.2 * col("bench_points"))
)

# rank the starter points against other teams in a given matchup week
window_spec = Window.partitionBy("roster._league_id","roster._matchup_week",).orderBy(F.asc("matchup_points"))
df_aggregated = df_aggregated.withColumn("starter_points_percentile", F.percent_rank().over(window_spec))

# rank all the players for a given position on a roster to understand strong players at each position
window_spec = Window.partitionBy("performance._league_id","performance._matchup_week", "performance.roster_id", "performance.player_position").orderBy(F.desc("performance.player_points"))
df_ranked = df_joined_results.withColumn("rank", F.rank().over(window_spec))

# get list of bench players
df_bench_better_than_starters = df_ranked.filter(
    ~col("performance.is_starter")
).select(
    "performance._league_id", "performance._matchup_week", "performance.roster_id", "performance.player_id",
    "performance.player_name", "performance.player_position", "performance.player_points"
).alias("bench")

# get list of starter players
df_starters = df_joined_results.filter(col("performance.is_starter")).select(
    "performance._league_id", "performance._matchup_week", "performance.roster_id", "performance.player_position",
    col("performance.player_name").alias("starter_player_name"),
    col("performance.player_points").alias("starter_player_points")
).alias('starters')

# join based on the matchup roster and player position to make suggestions about bench players that could have been starters
df_bench_better_than_starters = df_bench_better_than_starters.alias("bench").join(
    df_starters.alias("starters"),
    (col("bench._league_id") == col("starters._league_id")) &
    (col("bench._matchup_week") == col("starters._matchup_week")) &
    (col("bench.roster_id") == col("starters.roster_id")) &
    (col("bench.player_position") == col("starters.player_position"))
).filter(
    (col("bench.player_points") > col("starters.starter_player_points"))
).withColumn(
    "point_oppertunity_cost",
    (col("bench.player_points") - col("starters.starter_player_points")).alias("point_opportunity_cost")
)

# dedup bench players who get recommended twice (take the highest point potential)
window_spec = Window.partitionBy("bench._league_id","bench._matchup_week", "bench.roster_id", "bench.player_id").orderBy(F.desc("point_oppertunity_cost"))
df_bench_better_than_starters = df_bench_better_than_starters.withColumn("bench_player_dedup", F.rank().over(window_spec)).filter("bench_player_dedup == 1")

# create array of struct to list each suggestion
df_bench_better_than_starters = df_bench_better_than_starters.select(
    col("bench._league_id"), col("bench._matchup_week"), col("bench.roster_id"),
    F.struct(
        col("bench.player_name").alias("benched_player_name"),
        col("bench.player_points").alias("benched_player_points"),
        col("starters.starter_player_name"), col("starters.starter_player_points"),
        (col("bench.player_points") - col("starters.starter_player_points")).alias("point_opportunity_cost")
    ).alias("bench_better_than_starter")
)

# collect list of bench better than starters and sum of their points (max points)
df_bench_better_than_starters_agg = df_bench_better_than_starters.groupBy(
    "_league_id", "_matchup_week", "roster_id"
).agg(
    F.collect_list("bench_better_than_starter").alias("bench_better_than_starters"),
    F.sum("bench_better_than_starter.point_opportunity_cost").alias("missed_starter_points")
)

# join back to aggregated roster data
df_aggregated = df_aggregated.alias("aggregated").join(
    df_bench_better_than_starters_agg.alias("bench_agg"),
    ["_league_id", "_matchup_week", "roster_id"], "left"
)

# get top 3 highest performing starters
window_spec_highest_scoring = Window.partitionBy("roster._league_id", "roster._matchup_week", "roster.roster_id").orderBy(F.desc("performance.player_points"))
df_highest_scoring = df_joined_results.filter(col("performance.is_starter")).withColumn("rank", F.row_number().over(window_spec_highest_scoring)).filter(col("rank") <= 3)

# create array of structs for top 3 highest scoring players
df_highest_scoring_agg = df_highest_scoring.groupBy("roster._league_id", "roster._matchup_week","roster.roster_id").agg(
    F.collect_list(
        F.struct(
            col("performance.player_name").alias("highest_scoring_player_name"),
            col("performance.player_points").alias("highest_scoring_player_points")
        )
    ).alias("highest_scoring_players")
)

# join back to roster data
df_aggregated = df_aggregated.join(
    df_highest_scoring_agg.alias("high_score_agg"),
    ["_league_id", "_matchup_week", "roster_id"], "left"
)

# get opponent points
df_opponent_points = df_aggregated.select(
    col("_league_id"), 
    col("_matchup_week"),
    col("matchup_id"), 
    col("roster_id").alias("opponent_roster_id"), 
    col("matchup_points").alias("opponent_starter_points"),
    col("team_name").alias("opponent_team_name")
)

# self join to get opponent points and determine if missed starter points is enough to make a difference in the matchup. Determine managers efficiency based on ratio of starter points / max points
df_aggregated = df_aggregated.alias("aggregated").join(
    df_opponent_points.alias("opponent"),
    (col("aggregated._league_id") == col("opponent._league_id")) &
    (col("aggregated._matchup_week") == col("opponent._matchup_week")) &
    (col("aggregated.matchup_id") == col("opponent.matchup_id")) &
    (col("aggregated.roster_id") != col("opponent.opponent_roster_id")),
    "left"
).withColumn(
    "couldve_won_with_missed_bench_points",
    F.when(col("matchup_points") > col("opponent_starter_points"), None)
    .when((col("matchup_points") + col("missed_starter_points")) > col("opponent_starter_points"), True)
    .otherwise(False)
).withColumn(
    "manager_efficiency",
    col("starter_points") / (col("starter_points") + F.coalesce(col("missed_starter_points"), F.lit(0)))
).withColumn(
    "matchup_outcome",
    F.when(col("matchup_points") > col("opponent_starter_points"), "Win")
    .when(col("matchup_points") == col("opponent_starter_points"), "Draw")
    .otherwise("Loss")
).withColumn(
    "matchup_outcome_type",
    F.when((col("matchup_outcome") == "Win") & (col("starter_points_percentile") < 0.5), "Lucky Win")
    .when((col("matchup_outcome") == "Win") & (col("starter_points_percentile") >= 0.5), "Deserving Win")
    .when((col("matchup_outcome") == "Loss") & (col("starter_points_percentile") < 0.5), "Deserving Loss")
    .when((col("matchup_outcome") == "Loss") & (col("starter_points_percentile") >= 0.5), "Unlucky Loss")
    .when((col("matchup_outcome") == "Draw") & (col("starter_points_percentile") < 0.5), "Low Scoring Draw")
    .when((col("matchup_outcome") == "Draw") & (col("starter_points_percentile") >= 0.5), "High Scoring Draw")
)

# generate percentile rank of the roster strength points to normalize the power points
window_spec_percentile = Window.partitionBy("aggregated._league_id", "aggregated._matchup_week").orderBy("roster_strength")

# compute power_points for the week based on roster strength percentile and manager efficiency
df_aggregated = df_aggregated.withColumn("roster_strength_percentile", F.percent_rank().over(window_spec_percentile)).withColumn(
    "week_power_points",
    (col("roster_strength_percentile") * 0.85) + (col("manager_efficiency") * 0.15)
)

return df_aggregated.select(
    "aggregated._league_id", "aggregated.matchup_id", "aggregated.roster_id", "aggregated.owner_id",
    "aggregated.owner_name", "aggregated.is_commissioner", "aggregated.team_name", "aggregated.team_streak",
    "aggregated.team_record", "aggregated.team_total_wins", "aggregated.team_total_losses", "aggregated.team_total_ties",
    "aggregated.team_total_fpts", "aggregated.team_total_fpts_against", "aggregated.team_waiver_budget_used",
    "aggregated.matchup_points", "starter_points", "starter_points_percentile", "bench_points", "bench_better_than_starters", "matchup_outcome", "matchup_outcome_type",
    "missed_starter_points", "couldve_won_with_missed_bench_points", "opponent_starter_points", "opponent_team_name", "highest_scoring_players",
    "roster_strength", "roster_strength_percentile", "manager_efficiency", "week_power_points",
    "aggregated._matchup_week", "aggregated._year"
)

In [0]:
%sql
DROP TABLE sleeper.gold_power_rankings

In [0]:
display(df_aggregated.filter(df_aggregated._matchup_week == 14).select(
    "owner_name",
    "roster_id",
    "highest_scoring_players",
    "bench_better_than_starters",
    "couldve_won_with_missed_bench_points"
))

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col

df_roster_results = spark.read.table('sleeper.silver_model_roster_results')
df_player_performances = spark.read.table('sleeper.silver_model_player_performances')

df_joined_results = df_roster_results.alias("roster").join(
    df_player_performances.alias("performance"),
    (col("roster.roster_id") == col("performance.roster_id")) &
    (col("roster.matchup_id") == col("performance.matchup_id")) &
    (col("roster._matchup_week") == col("performance._matchup_week"))
)

df_aggregated = df_joined_results.groupBy(
    "roster.matchup_id", "roster.roster_id", "roster.owner_id", "roster.owner_name",
    "roster.is_commissioner", "roster.team_name", "roster.matchup_points", "roster.team_streak",
    "roster.team_record", "roster.team_total_wins", "roster.team_total_losses", "roster.team_total_ties",
    "roster.team_total_fpts", "roster.team_total_fpts_against", "roster.team_waiver_budget_used",
    "roster.team_waiver_position", "roster._league_id", "roster._matchup_week", "roster._year"
).agg(
    F.sum(F.when(col("performance.is_starter"), col("performance.player_points")).otherwise(0)).alias("starter_points"),
    F.sum(F.when(~col("performance.is_starter"), col("performance.player_points")).otherwise(0)).alias("bench_points")
).withColumn(
    "roster_strength", col("starter_points") + (0.2 * col("bench_points"))
)

window_spec = Window.partitionBy("performance.matchup_id", "performance.roster_id", "performance.player_position").orderBy(F.desc("performance.player_points"))
df_ranked = df_joined_results.withColumn("rank", F.rank().over(window_spec))

df_bench_better_than_starters = df_ranked.filter(
    (col("rank") == 1) & (~col("performance.is_starter"))
).select(
    "performance.matchup_id", "performance.roster_id", "performance.player_id",
    "performance.player_name", "performance.player_position", "performance.player_points"
)

df_starters = df_joined_results.filter(col("performance.is_starter")).select(
    "performance.matchup_id", "performance.roster_id", "performance.player_position",
    col("performance.player_name").alias("starter_player_name"),
    col("performance.player_points").alias("starter_player_points")
)

df_bench_better_than_starters = df_bench_better_than_starters.alias("bench").join(
    df_starters.alias("starters"),
    (col("bench.matchup_id") == col("starters.matchup_id")) &
    (col("bench.roster_id") == col("starters.roster_id")) &
    (col("bench.player_position") == col("starters.player_position"))
).select(
    col("bench.matchup_id"), col("bench.roster_id"),
    F.struct(
        col("bench.player_name").alias("benched_player_name"),
        col("bench.player_points").alias("benched_player_points"),
        col("starters.starter_player_name"), col("starters.starter_player_points"),
        (col("bench.player_points") - col("starters.starter_player_points")).alias("point_opportunity_cost")
    ).alias("bench_better_than_starter")
)

df_bench_better_than_starters_agg = df_bench_better_than_starters.groupBy(
    "matchup_id", "roster_id"
).agg(
    F.collect_list("bench_better_than_starter").alias("bench_better_than_starters"),
    F.sum("bench_better_than_starter.point_opportunity_cost").alias("missed_starter_points")
)

df_aggregated = df_aggregated.alias("aggregated").join(
    df_bench_better_than_starters_agg.alias("bench_agg"),
    ["matchup_id", "roster_id"], "left"
)

window_spec_highest_scoring = Window.partitionBy("roster.matchup_id", "roster.roster_id").orderBy(F.desc("performance.player_points"))
df_highest_scoring = df_joined_results.withColumn("rank", F.row_number().over(window_spec_highest_scoring)).filter(col("rank") <= 3)

df_highest_scoring_agg = df_highest_scoring.groupBy("roster.matchup_id", "roster.roster_id").agg(
    F.collect_list(
        F.struct(
            col("performance.player_name").alias("highest_scoring_player_name"),
            col("performance.player_points").alias("highest_scoring_player_points")
        )
    ).alias("highest_scoring_players")
)

df_aggregated = df_aggregated.join(
    df_highest_scoring_agg.alias("high_score_agg"),
    ["matchup_id", "roster_id"], "left"
)

df_opponent_points = df_aggregated.select(
    col("matchup_id"), col("roster_id").alias("opponent_roster_id"), col("starter_points").alias("opponent_starter_points")
)

df_aggregated = df_aggregated.alias("aggregated").join(
    df_opponent_points.alias("opponent"),
    (col("aggregated.matchup_id") == col("opponent.matchup_id")) &
    (col("aggregated.roster_id") != col("opponent.opponent_roster_id")),
    "left"
).withColumn(
    "couldve_won_with_missed_bench_points",
    F.when(col("starter_points") > col("opponent_starter_points"), None)
    .when((col("starter_points") + col("missed_starter_points")) > col("opponent_starter_points"), True)
    .otherwise(False)
).withColumn(
    "manager_efficiency",
    col("starter_points") / (col("starter_points") + F.coalesce(col("missed_starter_points"), F.lit(0)))
)

window_spec_percentile = Window.partitionBy("aggregated._league_id", "aggregated._matchup_week").orderBy("roster_strength")
df_aggregated = df_aggregated.withColumn("roster_strength_percentile", F.percent_rank().over(window_spec_percentile)).withColumn(
    "week_power_points",
    (col("roster_strength_percentile") * 0.8) + (col("manager_efficiency") * 0.2)
)

df_selected = df_aggregated.select(
    "aggregated._league_id", "aggregated.matchup_id", "aggregated.roster_id", "aggregated.owner_id",
    "aggregated.owner_name", "aggregated.is_commissioner", "aggregated.team_name", "aggregated.team_streak",
    "aggregated.team_record", "aggregated.team_total_wins", "aggregated.team_total_losses", "aggregated.team_total_ties",
    "aggregated.team_total_fpts", "aggregated.team_total_fpts_against", "aggregated.team_waiver_budget_used",
    "aggregated.matchup_points", "starter_points", "bench_points", "bench_better_than_starters",
    "missed_starter_points", "couldve_won_with_missed_bench_points", "highest_scoring_players",
    "roster_strength", "roster_strength_percentile", "manager_efficiency", "week_power_points",
    "aggregated._matchup_week", "aggregated._year"
).dropDuplicates(["_league_id", "roster_id", "_matchup_week"])

display(df_selected)

In [0]:
%sql

SELECT * FROM sleeper.gold_weekly_performance_ranks

In [0]:
%sql

SELECT * FROM sleeper.gold_weekly_performance_ranks

In [0]:
%sql
SELECT * FROM sleeper.gold_power_rankings

In [0]:
%sql
SELECT 
  r.matchup_id, 
  r.roster_id, 
  r.owner_id, 
  r.owner_name,
  r.is_commissioner, 
  r.team_name, 
  r.matchup_points, 
  r.team_streak,
  r.team_record, 
  r.team_total_wins, 
  r.team_total_losses, 
  r.team_total_ties,
  r.team_total_fpts, 
  r.team_total_fpts_against, 
  r.team_waiver_budget_used,
  r.team_waiver_position, 
  r._league_id, 
  r._matchup_week, 
  r._year,
  SUM(CASE WHEN p.is_starter THEN p.player_points ELSE 0 END) AS starter_points,
  SUM(CASE WHEN NOT p.is_starter THEN p.player_points ELSE 0 END) AS bench_points,
  SUM(CASE WHEN p.is_starter THEN p.player_points ELSE 0 END) + (0.2 * SUM(CASE WHEN NOT p.is_starter THEN p.player_points ELSE 0 END)) AS roster_strength
FROM sleeper.silver_model_roster_results r
  LEFT JOIN sleeper.silver_model_player_performances p
    ON r.roster_id = p.roster_id
    AND r._league_id = p._league_id
    AND r._matchup_week = p._matchup_week
GROUP BY 
  r.matchup_id, 
  r.roster_id, 
  r.owner_id, 
  r.owner_name,
  r.is_commissioner, 
  r.team_name, 
  r.matchup_points, 
  r.team_streak,
  r.team_record, 
  r.team_total_wins, 
  r.team_total_losses, 
  r.team_total_ties,
  r.team_total_fpts, 
  r.team_total_fpts_against, 
  r.team_waiver_budget_used,
  r.team_waiver_position, 
  r._league_id, 
  r._matchup_week, 
  r._year

In [0]:
%sql

SELECT * FROM sleeper.gold_weekly_performance_ranks

# build exp decay method

In [0]:
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import lit, col, rank
from pyspark.sql.window import Window

src = spark.read.table('sleeper.gold_weekly_performance_ranks')

src = src.withColumn('power_rank_points', lit(None).cast(DoubleType())) \
    .withColumn('power_ranking', lit(None).cast(IntegerType())) \
    .withColumn('power_rank_change', lit(None).cast(IntegerType()))

if spark.catalog.tableExists('sleeper.gold_power_rankings'):
    target = spark.read.table('sleeper.gold_power_rankings')
else:
    target = spark.createDataFrame([], src.schema)

# Alias columns in target
target_alias = target.select(
    col("_league_id").alias("target_league_id"),
    col("_matchup_week").alias("target_matchup_week"),
    col("roster_id").alias("target_roster_id")
)

# Perform the join
src = src.join(
    target_alias,
    (src["_league_id"] == target_alias["target_league_id"]) &
    (src["_matchup_week"] == target_alias["target_matchup_week"]) &
    (src["roster_id"] == target_alias["target_roster_id"]),
    how="left_anti"
).drop("target_league_id", "target_matchup_week", "target_roster_id")

distinct_league_week = src.select("_league_id", "_matchup_week").distinct().orderBy("_league_id", "_matchup_week")

for row in distinct_league_week.collect():
    league_id = row["_league_id"]
    week = row["_matchup_week"]
    print(f"Processing league {league_id} week {week}")

    temp = src.filter((src["_league_id"] == league_id) & (src["_matchup_week"] == week))

    previous_week = week - 1
    previous_week_data = target.filter((target["_league_id"] == league_id) & (target["_matchup_week"] == previous_week)).alias("prev_week")

    if previous_week_data.count() > 0:
        previous_week_data = previous_week_data \
            .select(
                col('roster_id').alias('previous_roster_id'),
                col("power_rank_points").alias('previous_power_rank_points'),
                col("power_ranking").alias('previous_power_ranking')
            )
        temp = temp.join(previous_week_data, temp["roster_id"] == previous_week_data["previous_roster_id"], how="left")
        temp = temp.withColumn(
            "power_rank_points",
            (col("week_power_points") * 0.3) + (col("previous_power_rank_points") * 0.7)
        )
    else:
        temp = temp \
            .withColumn("power_rank_points", col("week_power_points")) \
            .withColumn("power_rank_change", lit(None))

    # Calculate power_ranking
    window_spec = Window.partitionBy("_league_id").orderBy(col("power_rank_points").desc())
    temp = temp.withColumn("power_ranking", rank().over(window_spec))

    if previous_week_data.count() > 0:
        temp = temp.withColumn(
            "power_rank_change",
            col('previous_power_ranking') - col('power_ranking')
        )

        temp = temp.drop('previous_power_rank_points').drop('previous_roster_id').drop('previous_power_ranking')

    # Ensure schemas are compatible
    for col_name, col_type in target.dtypes:
        if col_name in temp.columns:
            temp = temp.withColumn(col_name, temp[col_name].cast(col_type))

    target = target.union(temp)

display(target)

In [0]:
%sql

DROP TABLE sleeper.gold_power_rankings

In [0]:
%sql

SELECT 
  owner_name,
  is_commissioner,
  team_name,
  team_streak,
  team_total_wins,
  team_total_losses,
  team_total_ties,
  team_total_fpts,
  team_total_fpts_against,
  matchup_points,
  missed_starter_points,
  bench_better_than_starters,
  couldve_won_with_missed_bench_points,
  highest_scoring_players,
  power_ranking,
  power_rank_change
FROM sleeper.gold_power_rankings WHERE `_matchup_week` = 14

In [0]:
df = spark.sql("""
SELECT 
  owner_name,
  is_commissioner,
  team_name,
  team_streak,
  team_total_wins,
  team_total_losses,
  team_total_ties,
  team_total_fpts,
  team_total_fpts_against,
  matchup_points,
  missed_starter_points,
  bench_better_than_starters,
  couldve_won_with_missed_bench_points,
  highest_scoring_players,
  power_ranking,
  power_rank_change
FROM sleeper.gold_power_rankings WHERE `_matchup_week` = 14            
""")

df.printSchema()

In [0]:
df = spark.read.table('sleeper.silver_users_dim')
display(df)

In [0]:
from pyspark.sql.functions import col

df = spark.read.table('sleeper.bronze_drafts_picks')

df = df.select(
    'draft_id',
    'draft_slot',
    'is_keeper',
    'metadata.*',
    'pick_no',
    'picked_by',
    'player_id',
    'reactions',
    'roster_id',
    'round',
    '_year',
    '_league_id',
    '_matchup_week',
    '_ingested_ts'
)

display(df)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc, col

draft_picks_df = spark.read.table('sleeper.silver_drafts_picks_fact')

users_df = spark.read.table('sleeper.silver_users_dim')

window_spec = Window.partitionBy('_league_id', 'owner_id', '_matchup_week').orderBy(desc('__START_AT'))
users_df = users_df.withColumn('rank', row_number().over(window_spec)).filter(col('rank') == 1).drop('rank')

joined_df = draft_picks_df.alias('picks').join(
    users_df.alias('users'),
    (draft_picks_df['_league_id'] == users_df['_league_id']) &
    (draft_picks_df['_matchup_week'] == users_df['_matchup_week']) &
    (draft_picks_df['picked_by'] == users_df['owner_id']),
    'inner'
)

joined_df = joined_df.select(
    'draft_id',
    'owner_id',
    'owner_name',
    'team_name',
    'roster_id',
    'draft_slot',
    'round',
    'pick_no',
    'player_id',
    'first_name',
    'last_name',
    'team',
    'position',
    'years_exp',
    'injury_status',
    'picks._league_id',
    'picks._year',
    'picks._matchup_week',
    'picks._ingested_ts'

)

In [0]:
display(joined_df)

In [0]:
%sql

SELECT * FROM sleeper.gold_power_rankings ORDER BY roster_id DESC, _matchup_week ASC

In [0]:
def generate_recursive_cte_sql(
    source_table: str,
    start_week: int,
    end_week: int,
    decay_weights=(0.3, 0.7),
    output_table_name: str = "power_rank_cte_view"
) -> str:
    ctes = []

    # Week 1: base case
    ctes.append(f"""
    week_{start_week} AS (
        SELECT
            _league_id,
            _matchup_week,
            roster_id,
            week_power_points,
            week_power_points AS power_rank_points
        FROM {source_table}
        WHERE _matchup_week = {start_week}
    )
    """)

    # Week 2 to N: apply decay using previous week's result
    for week in range(start_week + 1, end_week + 1):
        prev_week = week - 1
        w_curr, w_prev = decay_weights
        cte = f"""
        week_{week} AS (
            SELECT
                curr._league_id,
                curr._matchup_week,
                curr.roster_id,
                curr.week_power_points,
                ({w_curr} * curr.week_power_points + {w_prev} * prev.power_rank_points) AS power_rank_points
            FROM {source_table} curr
            JOIN week_{prev_week} prev
              ON curr._league_id = prev._league_id
             AND curr.roster_id = prev.roster_id
             AND curr._matchup_week = {week}
        )
        """
        ctes.append(cte)

    # Final SELECT from all weeks unioned
    union_all = "\nUNION ALL\n".join([f"SELECT * FROM week_{w}" for w in range(start_week, end_week + 1)])

    sql = f"""
    CREATE OR REPLACE TEMP VIEW {output_table_name} AS
    WITH
    {',\n'.join(ctes)}
    {union_all}
    """

    return sql


In [0]:
sql_string = generate_recursive_cte_sql(
    source_table="sleeper.gold_weekly_performance_ranks",
    start_week=13,
    end_week=15,
    decay_weights=(0.3, 0.7),
    output_table_name="power_rank_cte_view"
)

# Run it in Spark
spark.sql(sql_string)

In [0]:
print(sql_string)

In [0]:
CREATE OR REPLACE TEMP VIEW power_rank_cte_view AS
WITH

week_13 AS (
    SELECT
        _league_id,
        _matchup_week,
        roster_id,
        week_power_points,
        week_power_points AS power_rank_points
    FROM sleeper.gold_weekly_performance_ranks
    WHERE _matchup_week = 13
)
,

    week_14 AS (
        SELECT
            curr._league_id,
            curr._matchup_week,
            curr.roster_id,
            curr.week_power_points,
            (0.3 * curr.week_power_points + 0.7 * prev.power_rank_points) AS power_rank_points
        FROM sleeper.gold_weekly_performance_ranks curr
        JOIN week_13 prev
            ON curr._league_id = prev._league_id
            AND curr.roster_id = prev.roster_id
            AND curr._matchup_week = 14
    )
    ,

    week_15 AS (
        SELECT
            curr._league_id,
            curr._matchup_week,
            curr.roster_id,
            curr.week_power_points,
            (0.3 * curr.week_power_points + 0.7 * prev.power_rank_points) AS power_rank_points
        FROM sleeper.gold_weekly_performance_ranks curr
        JOIN week_14 prev
            ON curr._league_id = prev._league_id
            AND curr.roster_id = prev.roster_id
            AND curr._matchup_week = 15
    )
    
SELECT * FROM week_13
UNION ALL
SELECT * FROM week_14
UNION ALL
SELECT * FROM week_15

In [0]:
%sql

SELECT * FROM sleeper.gold_power_rankings ORDER BY roster_id ASC, _matchup_week ASC

In [0]:
df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("todays_score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
   .withColumn("zip", F.explode(F.arrays_zip("day","raw_score","todays_score")))\
   .select("user", "zip.*")\
   .show(truncate=False)

In [0]:
from pyspark.sql.functions import col, expr, collect_list, arrays_zip, explode
import pyspark.sql.functions as F

df = spark.read.table('sleeper.gold_weekly_performance_ranks')

df = df\
  .groupBy("_league_id","roster_id").agg(F.collect_list("week_power_points").alias("week_power_points"),F.collect_list("_matchup_week").alias("_matchup_week"))\
    .withColumn("raw_score1", F.expr("""transform(week_power_points,(x,i)-> struct(x as raw,i as index))"""))\
    .withColumn("todays_score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                                ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
    .withColumn("zip", F.explode(F.arrays_zip("roster_id","_matchup_week","week_power_points","todays_score")))\
   .select("user", "zip.*")

In [0]:
from pyspark.sql.functions import col, expr, collect_list, arrays_zip, explode
import pyspark.sql.functions as F

df = spark.read.table('sleeper.gold_weekly_performance_ranks')

df = df.groupBy("_league_id", "roster_id").agg(
    F.collect_list("week_power_points").alias("week_power_points"),
    F.collect_list("_matchup_week").alias("_matchup_week"),
    F.collect_list("roster_id").alias("roster_id_list")
).withColumn(
    "raw_score1", 
    F.expr("""transform(week_power_points, (x, i) -> struct(x as raw, i as index))""")
).withColumn(
    "previous_week_score", 
    F.expr("""
            transform(
                raw_score1, 
                x-> 
                IF (x.index = 0, 
                    x.raw,
                    aggregate(
                        filter(
                            raw_score1,
                            z-> z.index + 1 <= x.index
                        ),
                        cast(0 as double),
                        (acc,y)->(acc*.7)+(y.raw *.3)
                    )
                )
            )""")\
).withColumn(
    "zip", 
    F.explode(F.arrays_zip("roster_id_list", "_matchup_week", "week_power_points", "previous_week_score"))
).select('zip.*')

display(df.filter('roster_id = 1'))

In [0]:
df = sqlContext.createDataFrame([
                                 (0, 'a', 1),
                                 (1, 'a', 1),
                                 (2, 'a', 1),
                                 (3, 'a', 1)],
    ['day', 'user', 'raw_score']
)

df = df\
  .groupBy("user").agg(F.collect_list("raw_score").alias("raw_score"),F.collect_list("day").alias("day"))\
   .withColumn("raw_score1", F.expr("""transform(raw_score,(x,i)-> struct(x as raw,i as index))"""))\
   .withColumn("todays_score", F.expr("""transform(raw_score1, x-> aggregate(filter(raw_score1,z-> z.index<=x.index)\
                                             ,cast(0 as double),(acc,y)->(acc*0.9)+y.raw))"""))\
   .withColumn("zip", F.explode(F.arrays_zip("day","raw_score","todays_score")))\
   .select("user", "zip.*")

display(df)

In [0]:
display(spark.read.table('sleeper.gold_weekly_performance_ranks'))

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Adjust schema to match expected output exactly
schema = StructType([
    StructField("_league_id", StringType(), True),
    StructField("roster_id", IntegerType(), True),
    StructField("_matchup_week", IntegerType(), True),
    StructField("week_power_points", DoubleType(), True),
    StructField("previous_power_rank_score", DoubleType(), True),
    StructField("power_rank_score", DoubleType(), True)
])

pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)

def power_rankings(pdf):
    pdf = pdf.sort_values("_matchup_week").reset_index(drop=True)

    prscore_list = []
    prev_prscore_list = []

    for i, row in pdf.iterrows():
        if i == 0:
            prscore = row["week_power_points"]
            prev_prscore = None
        else:
            prscore = prev_prscore * 0.7 + row["week_power_points"] * 0.3
        
        prscore_list.append(prscore)
        prev_prscore_list.append(prev_prscore)
        prev_prscore = prscore

    pdf["power_rank_score"] = prscore_list
    pdf["previous_power_rank_score"] = prev_prscore_list

    return pdf[["_league_id", "roster_id", "_matchup_week", "week_power_points", "power_rank_score", "previous_power_rank_score"]]

df = spark.read.table('sleeper.gold_weekly_performance_ranks')

# Apply the function
power_rank_df = df.groupby("_league_id", "roster_id").applyInPandas(power_rankings, schema)

joined_df = df.alias('src').join(
    power_rank_df.alias('powerrank_df'), 
    on=["_league_id", "roster_id", "_matchup_week", "week_power_points"]
).select(
    "src.*", 
    "powerrank_df.power_rank_score", 
    "powerrank_df.previous_power_rank_score"
)

# Define a window over each league and week
rank_window = Window.partitionBy("_league_id", "_matchup_week").orderBy(F.desc("power_rank_score"))

# Add the power rank (1 = best score)
ranked_df = joined_df.withColumn("power_rank", F.dense_rank().over(rank_window))

prev_week_window = Window.partitionBy("_league_id", "roster_id").orderBy("_matchup_week")
final_df = ranked_df.withColumn(
    "previous_power_rank",
    F.lag("power_rank").over(prev_week_window)
)

display(final_df)


In [0]:
%sql

SELECT _matchup_week, roster_id, team_name, power_rank, power_rank_change, power_rank_score, week_power_points, previous_power_rank_score, previous_power_rank, roster_strength_percentile, manager_efficiency, * FROM sleeper.gold_power_rankings 
--WHERE `_matchup_week` = 13
ORDER BY roster_id, _matchup_week ASC